In [1]:
import os
import sys

# change to upper dir to use all custom libs (won't be needed if run from main scripts)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [2]:
import yadisk
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())
client = yadisk.Client(token=os.environ["YA_DISK_TOKEN"])

In [3]:
from scripts.helpers.yadisk_manager import download_model


experiments = {
    # a minor problem with <2lang> token where language group was not passed correctly and initial token == <unk>
#     'nllb.ru-mhr.both_direction.upd_tokenizer': "/bs-diploma/experiments/kaggle/ru-mhr.both_direction.mari-parallel.nllb.upd_tokenizer.finally_fixed",

    'nllb.ru-mht.both_direction.old_tokenizer': '/bs-diploma/experiments/kaggle/ru-mhr.both_directions.mari-parallel.nllb.no_upd_tokenizer',
    'nllb.ru-mhr.both_direction.upd_tokenizer': "/bs-diploma/experiments/kaggle/ru-mhr.both_directions.mari-parallel-413k.nllb.no_upd_tokenizer.mhr_fixed",
    'nllb.ru-mhr.sing_direction.upd_tokenizer': "/bs-diploma/experiments/kaggle/ru-mhr.single_direction.mari-parallel-413k.nllb.update_tokenizer",
    
    
    # TODO: rename to upd_tokenizer on ya.disk
    'nllb.ru-tat.both_direction.upd_tokenizer': '/bs-diploma/experiments/kaggle/ru-tat.both_directions.ipsan.tatar-parallel-400k.nllb.upd_tokenizer',
    'nllb.ru-tat.both_direction.old_tokenizer': '/bs-diploma/experiments/kaggle/ru-tt.both_direction.ipsan.nllb.base_tokenizer',
#     'nllb.ru-tat.sing_direction.old_tokenizer': '',  # TODO: add
#     'nllb.tat-ru.sing_direction.old_tokenizer': '',  # TODO: add
    
}


download_model(
    client=client,
    experiment=experiments['nllb.ru-mhr.both_direction.upd_tokenizer'],
    folder='models',
    checkpoint='last',
)

 ... (more hidden) ...
 ... (more hidden) ...
 ... (more hidden) ...


In [4]:
from scripts.helpers.model_manager import NLLB200Model

model = NLLB200Model.from_folder(
    model_path='models/model', 
    tokenizer_path='models/tokenizer', 
    convert_to_float16=True, 
    device='auto'
)

C:\Users\cutefluffyfox\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [14]:
trn_text = model.translate_single('hello worlds!', 'ru', 'mhr')
rus_text = model.translate_single(trn_text, 'mhr', 'ru')
rus_text, trn_text

('Здравствуйте!', 'Hello worlds!')

In [6]:
from scripts.benchmark.flores_dev import benchmark as benchmark_flores_dev

benchmark_flores_dev(model, 'ru-mhr')
benchmark_flores_dev(model, 'mhr-ru')

 ... (more hidden) ...
 ... (more hidden) ...


In [7]:
from scripts.helpers.metric_manager import BLEU, BLEURT, BertScore, CHRF, GoogleBLEU, FluencyRU

metrics = [
    BLEU(),
#     BLEURT(),
#     BertScore(),
    CHRF(),
#     FluencyRU(),
#     GoogleBLEU()
]

Using the latest cached version of the module from C:\Users\cutefluffyfox\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--bleu\9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Tue Nov  5 05:51:14 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.
Using the latest cached version of the module from C:\Users\cutefluffyfox\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--chrf\d244bab9383988714085a8dacc4871986d9f025398581c33d6b2ee22836b4069 (last modified on Tue Nov  5 05:54:58 2024) since it couldn't be found locally at evaluate-metric--chrf, or remotely on the Hugging Face Hub.


In [19]:
import pandas as pd
from scripts.helpers.path_manager import DataManager



dm = DataManager()
translations = [
    dm.get_path('benchmark', 'mhr-ru', 'mhr_both_dir.token_upd.mhr_ru.last.txt'),
    dm.get_path('benchmark', 'mhr-ru', 'mhr_both_dir.token_upd_broken.mhr_ru.last.txt'),
    dm.get_path('benchmark', 'mhr-ru', 'mhr_both_dir.token_upd_broken.mhr_ru.7200.txt'),
    dm.get_path('benchmark', 'mhr-ru', 'mhr_both_dir.old_tokenizer.mhr_ru.last.txt'),
    dm.get_path('benchmark', 'mhr-ru', 'nllb200.mhr_ru.txt'),
    dm.get_path('benchmark', 'mhr-ru', 'madlad400.mhr_ru.txt'),

    dm.get_path('benchmark', 'ru-mhr', 'mhr_both_dir.token_upd.ru_mhr.last.txt'),
    dm.get_path('benchmark', 'ru-mhr', 'mhr_single_dir.token_upd.ru_mhr.last.txt'),
    dm.get_path('benchmark', 'ru-mhr', 'mhr_both_dir.token_upd_broken.ru_mhr.last.txt'),
    dm.get_path('benchmark', 'ru-mhr', 'mhr_both_dir.token_upd_broken.ru_mhr.7200.txt'),
    dm.get_path('benchmark', 'ru-mhr', 'mhr_both_dir.old_tokenizer.ru_mhr.last.txt'),
    dm.get_path('benchmark', 'ru-mhr', 'madlad400.ru_mhr.txt'),

    
    dm.get_path('benchmark', 'tat-ru', 'tat_both_dir.no_token_upd.tat_ru.last.txt'),
    dm.get_path('benchmark', 'tat-ru', 'tat_both_dir.token_upd.tat_ru.last.txt'),
    dm.get_path('benchmark', 'tat-ru', 'nllb200.tat_ru.txt'),
    dm.get_path('benchmark', 'tat-ru', 'madlad400.tat_ru.txt'),

    dm.get_path('benchmark', 'ru-tat', 'tat_both_dir.no_token_upd.ru_tat.last.txt'),
    dm.get_path('benchmark', 'ru-tat', 'tat_both_dir.token_upd.ru_tat.last.txt'),
    dm.get_path('benchmark', 'ru-tat', 'nllb200.ru_tat.txt'),
    dm.get_path('benchmark', 'ru-tat', 'madlad400.ru_tat.txt'),

    dm.get_path('benchmark', 'kaz-ru', 'nllb200.kaz_ru.txt'),
    dm.get_path('benchmark', 'kaz-ru', 'madlad400.kaz_ru.txt'),

    
    dm.get_path('benchmark', 'ru-kaz', 'nllb200.ru_kaz.txt'),
    dm.get_path('benchmark', 'ru-kaz', 'madlad400.ru_kaz.txt'),
]


prev_dir = ''
for trans in translations:
    direction = trans.split('benchmark')[-1].replace('\\', '/').strip('/').split('/')[0]
    if prev_dir != direction:
        print('\n\n\n')
        prev_dir = direction
    
    print('==========', trans.split('benchmark')[-1], '==========')
    src_lang, dst_lang = direction.split('-')
    for metric in metrics:
        if not metric.lang_is_supported(metric):
            continue
        
        with open(trans, 'r', encoding='UTF-8') as file:
            res = [eval(line.strip()) for line in file.readlines()]
        df = pd.DataFrame(res)
        if dst_lang == 'ru' and 'ru' not in df.columns and 'rus' in df.columns:
            dst_lang = 'rus'
        res_trans, res_ref = df['translation'].to_list(), df[dst_lang].to_list()

        print(metric.metric_name, ':', metric(
            sources=None,
            targets=res_ref,
            translations=res_trans,
            lang=dst_lang
        ))





========== \mhr-ru\mhr_both_dir.token_upd.mhr_ru.last.txt ==========
bleu : 0.1573674741106463
chrf : 45.86165017933309
========== \mhr-ru\mhr_both_dir.token_upd_broken.mhr_ru.last.txt ==========
bleu : 0.08968501201433417
chrf : 36.77403867823899
========== \mhr-ru\mhr_both_dir.token_upd_broken.mhr_ru.7200.txt ==========
bleu : 0.12787250706142736
chrf : 43.42382341215098
========== \mhr-ru\mhr_both_dir.old_tokenizer.mhr_ru.last.txt ==========
bleu : 0.009983102617198474
chrf : 16.366378658826093
========== \mhr-ru\nllb200.mhr_ru.txt ==========
bleu : 0.03707498445596439
chrf : 27.610867511783933
========== \mhr-ru\madlad400.mhr_ru.txt ==========
bleu : 0.14416953753714376
chrf : 38.20947588930588




========== \ru-mhr\mhr_both_dir.token_upd.ru_mhr.last.txt ==========
bleu : 0.013422333820341265
chrf : 14.586913117507091
========== \ru-mhr\mhr_single_dir.token_upd.ru_mhr.last.txt ==========
bleu : 0.0882602692724267
chrf : 42.27905735258935
========== \ru-mhr\mhr_both_dir.token_u

In [30]:
with open('C:\\Users\\cutefluffyfox\\PycharmProjects\\MT-vocab-tuning\\data\\benchmark\\mhr-ru\\nllb200.mhr_ru.txt', 'r', encoding='UTF-8') as file:
    print(eval(file.readlines()[942])['translation'])